# LHL Mini Project 2 - Locations API

### Import Libraries

In [1]:
import requests as re
import os
import numpy as np
import pandas as pd
from IPython.display import JSON
import folium
from geopy import distance

### Set variables

In [2]:
#Yelp API Variables
y_API_key = os.environ["YELP_AK"]
y_root_endpoint = 'https://api.yelp.com/v3/'
y_auth = f'Bearer {y_API_key}'
y_head = {'Authorization' : y_auth}

In [3]:
#FourSquare API Variables
fs_client_id = os.environ["FSQ_CI"]
fs_client_secret = os.environ["FSQ_CS"]
fs_root_endpoint = 'https://api.foursquare.com/v2/'
fs_auth = f'&client_id={fs_client_id}&client_secret={fs_client_secret}&v=20200731'

In [4]:
# define latitude and longitude of target in Toronto
tor_ll = {'latitude' : '43.65904317560671',
          'longitude' : '-79.44004997396817',
          'll' : '43.65904317560671,-79.44004997396817'
         }

In [5]:
# define latitude and longitude of target in Tokyo
tokyo_ll = {'latitude' : '35.565734606428144',
          'longitude' : '139.73789670131674',
            'll' : '35.565734606428144,139.73789670131674'
         }

### Define function to call API GET

In [6]:
# Function to call API GET
def api_get(api_name, api_method, headers='', params=''):
    
    # define endpoint url
    if api_name == 'fs':
        endpoint = fs_root_endpoint + api_method + fs_auth
    elif api_name == 'y':
        endpoint = y_root_endpoint + api_method
    else:
        print('no such api_name!')
        
    # send request to API and store response
    response = re.get(endpoint,
                      headers=headers,
                      params=params
                     )
    
    # print error code
    print(response.status_code)
    
    # save and return response as a JSON file
    result = response.json()
    return result

### Call API

In [7]:
# establish parameters to pass
fs_tor_par = {'ll' : tor_ll['ll'],
              'limit' : 50,
              'radius' : 1000
             }

re_tor_fs = api_get('fs', 'venues/explore?', params=fs_tor_par)

200


In [8]:
# establish parameters to pass
fs_tokyo_par = {'ll' : tokyo_ll['ll'],
                'limit' : 50,
                'radius' : 1000
               }

re_tokyo_fs = api_get('fs', 'venues/explore?', params=fs_tokyo_par)

200


In [9]:
# establish parameters to pass
y_tor_par = {'latitude' : tor_ll['latitude'],
             'longitude' : tor_ll['longitude'],
             'limit' : 50,
             'radius' : 1000
            }

re_tor_y = api_get('y', 'businesses/search', y_head, y_tor_par)

200


In [10]:
# establish parameters to pass
y_tokyo_par = {'latitude' : tokyo_ll['latitude'],
               'longitude' : tokyo_ll['longitude'],
               'limit' : 50,
               'radius' : 1000
              }

re_tokyo_y = api_get('y', 'businesses/search', y_head, y_tokyo_par)

200


### Convert results to Pandas DataFrames

In [11]:
# establish dataframes from results and print first 5 lines of each
df_tor_fs = pd.json_normalize(re_tor_fs, record_path=['response','groups','items'])
df_tor_fs.head()

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,...,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.location.crossStreet,venue.location.neighborhood
0,e-0-55208e01498e31eeac5ff582-0,0,"[{'summary': 'This spot is popular', 'type': '...",55208e01498e31eeac5ff582,Burdock,1184 Bloor St W,43.659470,-79.437763,"[{'label': 'display', 'lat': 43.65947030669613...",190,...,CA,Toronto,ON,Canada,"[1184 Bloor St W, Toronto ON M6H 1N2, Canada]","[{'id': '56aa371ce4b08b9a8d57356c', 'name': 'B...",0,[],NaN,NaN
1,e-0-4adb4489f964a520642521e3-1,0,"[{'summary': 'This spot is popular', 'type': '...",4adb4489f964a520642521e3,Three Speed,1163 Bloor St W,43.659293,-79.438025,"[{'label': 'display', 'lat': 43.6592927979752,...",165,...,CA,Toronto,ON,Canada,"[1163 Bloor St W (btw Dufferin & Brock), Toron...","[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",0,[],btw Dufferin & Brock,NaN
2,e-0-51f7efd4498ed61c0dd98f55-2,0,"[{'summary': 'This spot is popular', 'type': '...",51f7efd4498ed61c0dd98f55,Brock Sandwich,1260 Bloor St. W.,43.658846,-79.440521,"[{'label': 'display', 'lat': 43.65884589365906...",43,...,CA,Toronto,ON,Canada,"[1260 Bloor St. W. (at Emerson Ave.), Toronto ...","[{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...",0,[],at Emerson Ave.,NaN
3,e-0-57bde1d0cd10f990bb59e644-3,0,"[{'summary': 'This spot is popular', 'type': '...",57bde1d0cd10f990bb59e644,Baddies,679 Lansdowne Ave,43.658690,-79.442793,"[{'label': 'display', 'lat': 43.65868980411702...",224,...,CA,Toronto,ON,Canada,"[679 Lansdowne Ave (Bloor & Lansdowne), Toront...","[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",0,[],Bloor & Lansdowne,NaN
4,e-0-5963b3e52be425495a1ed8a1-4,0,"[{'summary': 'This spot is popular', 'type': '...",5963b3e52be425495a1ed8a1,Sugo,1281 Bloor St. W.,43.658412,-79.442443,"[{'label': 'display', 'lat': 43.65841172348161...",205,...,CA,Toronto,ON,Canada,"[1281 Bloor St. W. (Lansdowne), Toronto ON M6H...","[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",0,[],Lansdowne,Brockton Village


In [12]:
df_tokyo_fs = pd.json_normalize(re_tokyo_fs, record_path=['response','groups','items'])
df_tokyo_fs.head()

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,...,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.location.postalCode,venue.location.crossStreet,venue.location.neighborhood
0,e-0-4d1803ba6d1037049ae33bbd-0,0,"[{'summary': 'This spot is popular', 'type': '...",4d1803ba6d1037049ae33bbd,locofee ロコフィ,大森西3-25-5,35.571954,139.732114,"[{'label': 'display', 'lat': 35.57195400031886...",868,...,東京,東京都,日本,"[大森西3-25-5, 大田区, 東京都, 日本]","[{'id': '4bf58dd8d48988d16c941735', 'name': 'B...",0,[],NaN,NaN,NaN
1,e-0-50021a9fe4b0ed54498a8615-1,0,"[{'summary': 'This spot is popular', 'type': '...",50021a9fe4b0ed54498a8615,Ota City General Gymnasium (大田区総合体育館),東蒲田1-11-1,35.564600,139.727776,"[{'label': 'display', 'lat': 35.56459966644509...",925,...,大田区,東京都,日本,"[東蒲田1-11-1, 大田区, 東京都, 144-0031, 日本]","[{'id': '4bf58dd8d48988d184941735', 'name': 'S...",0,[],144-0031,NaN,NaN
2,e-0-5e137af6697f0a00086742f6-2,0,"[{'summary': 'This spot is popular', 'type': '...",5e137af6697f0a00086742f6,セリア,NaN,35.564505,139.742551,"[{'label': 'display', 'lat': 35.564505, 'lng':...",443,...,大田区,東京都,日本,"[大田区, 東京都, 143-0013, 日本]","[{'id': '52dea92d3cf9994f4e043dbb', 'name': 'D...",0,[],143-0013,NaN,NaN
3,e-0-4f818bd2e4b0e160ba4cc7b7-3,0,"[{'summary': 'This spot is popular', 'type': '...",4f818bd2e4b0e160ba4cc7b7,アコーディア・ガーデン東京ベイ,大森東3-28-1,35.573550,139.738980,"[{'label': 'display', 'lat': 35.57355, 'lng': ...",875,...,大田区,東京都,日本,"[大森東3-28-1, 大田区, 東京都, 143-0012, 日本]","[{'id': '58daa1558bbb0b01f18ec1b0', 'name': 'G...",0,[],143-0012,NaN,NaN
4,e-0-52cb7ff7498e6fd18d133c05-4,0,"[{'summary': 'This spot is popular', 'type': '...",52cb7ff7498e6fd18d133c05,FamilyMart (ファミリーマート 三度屋大森東四丁目店),大森東4-17-1,35.568310,139.737448,"[{'label': 'display', 'lat': 35.56830981126114...",289,...,大田区,東京都,日本,"[大森東4-17-1, 大田区, 東京都, 143-0012, 日本]","[{'id': '4d954b0ea243a5684a65b473', 'name': 'C...",0,[],143-0012,NaN,NaN


In [13]:
df_tor_y = pd.json_normalize(re_tor_y, record_path=['businesses'])
df_tor_y.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,coordinates.latitude,coordinates.longitude,location.address1,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address
0,zEwQJl9haP5BYIl0peOpGA,sugo-toronto,Sugo,https://s3-media2.fl.yelpcdn.com/bphoto/5Fykw9...,False,https://www.yelp.com/biz/sugo-toronto?adjust_c...,199,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,[],...,43.658250,-79.442270,1281 Bloor Street W,,None,Toronto,M6H 1N7,CA,ON,"[1281 Bloor Street W, Toronto, ON M6H 1N7, Can..."
1,7DW2fsInJRfOHMTj2Z6fJw,brock-sandwich-toronto,Brock Sandwich,https://s3-media2.fl.yelpcdn.com/bphoto/7htxIK...,False,https://www.yelp.com/biz/brock-sandwich-toront...,101,"[{'alias': 'salad', 'title': 'Salad'}, {'alias...",4.5,[],...,43.658906,-79.440578,1260 Bloor Street W,,,Toronto,M6H 1N5,CA,ON,"[1260 Bloor Street W, Toronto, ON M6H 1N5, Can..."
2,bbSb-uviEUNlCw1bitAmQg,baddies-toronto-2,Baddies,https://s3-media3.fl.yelpcdn.com/bphoto/amGibO...,False,https://www.yelp.com/biz/baddies-toronto-2?adj...,85,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.0,[],...,43.658801,-79.442668,679 Lansdowne Avenue,,None,Toronto,M6H 3Y9,CA,ON,"[679 Lansdowne Avenue, Toronto, ON M6H 3Y9, Ca..."
3,ovnqQaQ8BsPgDoJ4nlYBUw,bakerbots-baking-toronto,Bakerbots Baking,https://s3-media1.fl.yelpcdn.com/bphoto/dQvRP2...,False,https://www.yelp.com/biz/bakerbots-baking-toro...,293,"[{'alias': 'desserts', 'title': 'Desserts'}, {...",4.0,[],...,43.661710,-79.427860,205 Delaware Avenue,,,Toronto,M6H 2T4,CA,ON,"[205 Delaware Avenue, Toronto, ON M6H 2T4, Can..."
4,H9pG0BNhqID2MTj8KnY3jQ,coo-café-bread-or-rice-toronto,COO Café Bread or Rice,https://s3-media4.fl.yelpcdn.com/bphoto/ZYXsQ1...,False,https://www.yelp.com/biz/coo-caf%C3%A9-bread-o...,84,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.5,[],...,43.660497,-79.431928,1049 Bloor Street W,,,Toronto,M6H 1M4,CA,ON,"[1049 Bloor Street W, Toronto, ON M6H 1M4, Can..."


In [14]:
df_tokyo_y = pd.json_normalize(re_tokyo_y, record_path=['businesses'])
df_tokyo_y.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,coordinates.longitude,location.address1,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address,price
0,n-Zay_vKhhOEFQjOF1B7Mg,大森ふるさとの浜辺公園-大田区,Oomori furusatonohamabe kouen,https://s3-media2.fl.yelpcdn.com/bphoto/R6dpQF...,False,https://www.yelp.com/biz/%E5%A4%A7%E6%A3%AE%E3...,5,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,[],...,139.741485,ふるさとの浜辺公園1-1,,,Ōta,,JP,13,"[ふるさとの浜辺公園1-1, Ōta, 東京都, Japan]",NaN
1,DtbufaMUdrX9pinKoH2Mqg,プリンチペ-デル-ソーレ-大田区,PRINCIPE del sole,https://s3-media2.fl.yelpcdn.com/bphoto/ahQhAC...,False,https://www.yelp.com/biz/%E3%83%97%E3%83%AA%E3...,2,"[{'alias': 'pizza', 'title': 'Pizza'}]",5.0,[],...,139.729162,東蒲田1-1-3,宝ビル 1F,,Ōta,144-0031,JP,13,"[東蒲田1-1-3, 宝ビル 1F, Ōta, 東京都 〒144-0031, Japan]",NaN
2,EuY4aWt9sOrFoBFdxZfuSg,満願堂-羽田空港店-大田区,Mangan-dō Haneda Airport,https://s3-media2.fl.yelpcdn.com/bphoto/bBH1l5...,False,https://www.yelp.com/biz/%E6%BA%80%E9%A1%98%E5...,1,"[{'alias': 'jpsweets', 'title': 'Japanese Swee...",5.0,[],...,139.742057,羽田空港3-4-2,第2旅客ターミナル 2F,,Ōta,144-0041,JP,13,"[羽田空港3-4-2, 第2旅客ターミナル 2F, Ōta, 東京都 〒144-0041, ...",￥￥
3,9_wSU7tFxpJSJeiLY_Ym4Q,貴船掘公園-大田区,貴船掘公園,https://s3-media1.fl.yelpcdn.com/bphoto/mNCWnQ...,False,https://www.yelp.com/biz/%E8%B2%B4%E8%88%B9%E6...,1,"[{'alias': 'parks', 'title': 'Parks'}]",5.0,[],...,139.741205,大森東3丁目,,,Ōta,143-0012,JP,13,"[大森東3丁目, Ōta, 東京都 〒143-0012, Japan]",NaN
4,mWR5xjHZyMB4L4_KrdtNZg,たぐい食堂-大田区,Taguishokudō,https://s3-media1.fl.yelpcdn.com/bphoto/orNn-d...,False,https://www.yelp.com/biz/%E3%81%9F%E3%81%90%E3...,2,"[{'alias': 'onigiri', 'title': 'Onigiri'}]",4.5,[],...,139.731644,大森西3-21-5,,,Ōta,143-0015,JP,13,"[大森西3-21-5, Ōta, 東京都 〒143-0015, Japan]",NaN


### Get FourSquare review count / rating

In [15]:
# Oh no, it's a premium API!

### Build Map of Toronto

In [16]:
# initialize map of Toronto with Apartment1 marker

tor_map = folium.Map(location=[tor_ll['latitude'], tor_ll['longitude']], zoom_start=15, tiles='Stamen Toner')

folium.Marker(
    location=[tor_ll['latitude'], tor_ll['longitude']],
    icon=folium.Icon(color="purple"),
    popup="Apartment 1"
).add_to(tor_map)

In [17]:
# Add markers for FourSquare results

for lat, lng, label in zip(df_tor_fs['venue.location.lat'], df_tor_fs['venue.location.lng'], df_tor_fs['venue.name']):
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color="blue"),
        popup=f'FourSquare: {label}'
    ).add_to(tor_map)

In [18]:
# Add markers for Yelp results

for lat, lng, label in zip(df_tor_y['coordinates.latitude'], df_tor_y['coordinates.longitude'], df_tor_y.name):
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color="red"),
        popup=f'Yelp: {label}'
    ).add_to(tor_map)

In [19]:
# Save map to file

tor_map.save(outfile='torontomap.html')

In [20]:
tor_map

### Build Map of Tokyo

In [21]:
# initialize map of Tokyo with Apartment2 marker

tokyo_map = folium.Map(location=[tokyo_ll['latitude'], tokyo_ll['longitude']], zoom_start=15, tiles='Stamen Toner')

folium.Marker(
    location=[tokyo_ll['latitude'], tokyo_ll['longitude']],
    icon=folium.Icon(color="purple"),
    popup="Apartment 1"
).add_to(tokyo_map)

In [22]:
# Add markers for FourSquare results

for lat, lng, label in zip(df_tokyo_fs['venue.location.lat'], df_tokyo_fs['venue.location.lng'], df_tokyo_fs['venue.name']):
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color="blue"),
        popup=f'FourSquare: {label}'
    ).add_to(tokyo_map)

In [23]:
# Add markers for Yelp results

for lat, lng, label in zip(df_tokyo_y['coordinates.latitude'], df_tokyo_y['coordinates.longitude'], df_tokyo_y.name):
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color="red"),
        popup=f'Yelp: {label}'
    ).add_to(tokyo_map)

In [24]:
# Save map to file

tokyo_map.save(outfile='tokyomap.html')

In [25]:
tokyo_map

### Save Data to SQLite using sqlalchemy

In [26]:
df_foursquare = pd.concat([df_tokyo_fs, df_tor_fs])
df_yelp = pd.concat([df_tokyo_y, df_tor_y])

In [27]:
df_foursquare = df_foursquare.astype('str')

In [28]:
df_foursquare.dtypes

referralId                         object
reasons.count                      object
reasons.items                      object
venue.id                           object
venue.name                         object
venue.location.address             object
venue.location.lat                 object
venue.location.lng                 object
venue.location.labeledLatLngs      object
venue.location.distance            object
venue.location.cc                  object
venue.location.city                object
venue.location.state               object
venue.location.country             object
venue.location.formattedAddress    object
venue.categories                   object
venue.photos.count                 object
venue.photos.groups                object
venue.location.postalCode          object
venue.location.crossStreet         object
venue.location.neighborhood        object
dtype: object

In [29]:
df_yelp = df_yelp.astype('str')

In [30]:
df_yelp.dtypes

id                          object
alias                       object
name                        object
image_url                   object
is_closed                   object
url                         object
review_count                object
categories                  object
rating                      object
transactions                object
phone                       object
display_phone               object
distance                    object
coordinates.latitude        object
coordinates.longitude       object
location.address1           object
location.address2           object
location.address3           object
location.city               object
location.zip_code           object
location.country            object
location.state              object
location.display_address    object
price                       object
dtype: object

In [31]:
from sqlalchemy import create_engine

In [32]:
engine = create_engine('sqlite:///POI.db', echo=True)
sqlite_connection = engine.connect()

In [33]:
sqlite_table = "Yelp"
df_yelp.to_sql(sqlite_table, sqlite_connection, if_exists='replace')

2021-11-06 09:57:19,530 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Yelp")
2021-11-06 09:57:19,532 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-11-06 09:57:19,533 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Yelp")
2021-11-06 09:57:19,533 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-11-06 09:57:19,534 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-11-06 09:57:19,535 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-11-06 09:57:19,535 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("Yelp")
2021-11-06 09:57:19,536 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-11-06 09:57:19,538 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2021-11-06 09:57:19,557 INFO sqlalchemy.engine.Engine [raw sql] ('Yelp',)
2021-11-06 09:57:19,559 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("Yelp")
2021-11-06 09

In [34]:
sqlite_table = "FourSquare"
df_foursquare.to_sql(sqlite_table, sqlite_connection, if_exists='replace')

2021-11-06 09:57:19,603 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("FourSquare")
2021-11-06 09:57:19,604 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-11-06 09:57:19,605 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("FourSquare")
2021-11-06 09:57:19,605 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-11-06 09:57:19,607 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-11-06 09:57:19,607 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-11-06 09:57:19,609 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("FourSquare")
2021-11-06 09:57:19,610 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-11-06 09:57:19,614 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2021-11-06 09:57:19,614 INFO sqlalchemy.engine.Engine [raw sql] ('FourSquare',)
2021-11-06 09:57:19,615 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_li

### Analyze Data

#### average distance from location

In [35]:
# initialize empty lists
tor_dist_fs = []
tor_dist_y = []
tokyo_dist_fs = []
tokyo_dist_y = []

In [36]:
# iterate thru the row items for Toronto results
for idx in range(len(df_tor_fs)):
    
    # collect distances for FourSquare results
    ll_fs = [df_tor_fs.iloc[idx]['venue.location.lat'], df_tor_fs.iloc[idx]['venue.location.lng']]
    tor_dist_fs.append(distance.distance(tor_ll['ll'], ll_fs).m)

    # collect distances for Yelp results
    ll_y = [df_tor_y.iloc[idx]['coordinates.latitude'], df_tor_y.iloc[idx]['coordinates.longitude']]
    tor_dist_y.append(distance.distance(tor_ll['ll'], ll_y).m)

In [37]:
# Display results for Toronto
avg_dist_tor_y = pd.Series(tor_dist_y).mean()
avg_dist_tor_fs = pd.Series(tor_dist_fs).mean()
avg_dist_tor = pd.Series([avg_dist_tor_y, avg_dist_tor_fs]).mean()


print(f'The average distance from home for POI returned by Yelp in Toronto is: {avg_dist_tor_y} meters.')
print(f'The average distance from home for POI returned by FourSquare in Toronto is: {avg_dist_tor_fs} meters.')
print(f'The average distance from home for POI returned by both Yelp and FourSquare in Toronto is: {avg_dist_tor} meters.')

The average distance from home for POI returned by Yelp in Toronto is: 623.3761171989673 meters.
The average distance from home for POI returned by FourSquare in Toronto is: 507.1125301890463 meters.
The average distance from home for POI returned by both Yelp and FourSquare in Toronto is: 565.2443236940069 meters.


In [38]:
# iterate thru the row items for Tokyo results
for idx in range(len(df_tokyo_fs)):
    
    # collect distances for FourSquare results
    ll_fs = [df_tokyo_fs.iloc[idx]['venue.location.lat'], df_tokyo_fs.iloc[idx]['venue.location.lng']]
    tokyo_dist_fs.append(distance.distance(tokyo_ll['ll'], ll_fs).m)

    # collect distances for Yelp results
    try:
        ll_y = [df_tokyo_y.iloc[idx]['coordinates.latitude'], df_tokyo_y.iloc[idx]['coordinates.longitude']]
        tokyo_dist_y.append(distance.distance(tokyo_ll['ll'], ll_y).m)
    except:
        pass

In [39]:
# Display results for Toronto
avg_dist_tokyo_y = pd.Series(tokyo_dist_y).mean()
avg_dist_tokyo_fs = pd.Series(tokyo_dist_fs).mean()
avg_dist_tokyo = pd.Series([avg_dist_tokyo_y, avg_dist_tokyo_fs]).mean()


print(f'The average distance from home for POI returned by Yelp in Tokyo is: {avg_dist_tokyo_y} meters.')
print(f'The average distance from home for POI returned by FourSquare in Tokyo is: {avg_dist_tokyo_fs} meters.')
print(f'The average distance from home for POI returned by both Yelp and FourSquare in Tokyto is: {avg_dist_tokyo} meters.')

The average distance from home for POI returned by Yelp in Tokyo is: 766.5763179410781 meters.
The average distance from home for POI returned by FourSquare in Tokyo is: 696.9069885825243 meters.
The average distance from home for POI returned by both Yelp and FourSquare in Tokyto is: 731.7416532618012 meters.


#### Number of Results returned

In [40]:
print(len(df_tokyo_fs))
print(len(df_tokyo_y))
print(len(df_tor_fs))
print(len(df_tor_y))


50
20
50
50


#### Blank Entries

In [41]:
df_tokyo_y[df_tokyo_y == ''].count()

id                          0
alias                       0
name                        0
image_url                   2
is_closed                   0
url                         0
review_count                0
categories                  0
rating                      0
transactions                0
phone                       4
display_phone               4
distance                    0
coordinates.latitude        0
coordinates.longitude       0
location.address1           0
location.address2           9
location.address3           9
location.city               0
location.zip_code           1
location.country            0
location.state              0
location.display_address    0
price                       0
dtype: int64

In [42]:
df_tokyo_fs[df_tokyo_fs == ''].count()

referralId                         0
reasons.count                      0
reasons.items                      0
venue.id                           0
venue.name                         0
venue.location.address             0
venue.location.lat                 0
venue.location.lng                 0
venue.location.labeledLatLngs      0
venue.location.distance            0
venue.location.cc                  0
venue.location.city                0
venue.location.state               0
venue.location.country             0
venue.location.formattedAddress    0
venue.categories                   0
venue.photos.count                 0
venue.photos.groups                0
venue.location.postalCode          0
venue.location.crossStreet         0
venue.location.neighborhood        0
dtype: int64

In [43]:
df_tor_y[df_tor_y == ''].count()

id                           0
alias                        0
name                         0
image_url                    0
is_closed                    0
url                          0
review_count                 0
categories                   0
rating                       0
transactions                 0
price                        0
phone                        5
display_phone                5
distance                     0
coordinates.latitude         0
coordinates.longitude        0
location.address1            0
location.address2           38
location.address3           43
location.city                0
location.zip_code            0
location.country             0
location.state               0
location.display_address     0
dtype: int64

In [44]:
df_tor_fs[df_tor_fs == ''].count()

referralId                         0
reasons.count                      0
reasons.items                      0
venue.id                           0
venue.name                         0
venue.location.address             0
venue.location.lat                 0
venue.location.lng                 0
venue.location.labeledLatLngs      0
venue.location.distance            0
venue.location.postalCode          0
venue.location.cc                  0
venue.location.city                0
venue.location.state               0
venue.location.country             0
venue.location.formattedAddress    0
venue.categories                   0
venue.photos.count                 0
venue.photos.groups                0
venue.location.crossStreet         0
venue.location.neighborhood        0
dtype: int64

#### Top 10

In [45]:
df_tor_y.head(1)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,coordinates.latitude,coordinates.longitude,location.address1,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address
0,zEwQJl9haP5BYIl0peOpGA,sugo-toronto,Sugo,https://s3-media2.fl.yelpcdn.com/bphoto/5Fykw9...,False,https://www.yelp.com/biz/sugo-toronto?adjust_c...,199,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,[],...,43.65825,-79.44227,1281 Bloor Street W,,None,Toronto,M6H 1N7,CA,ON,"[1281 Bloor Street W, Toronto, ON M6H 1N7, Can..."


In [46]:
df_tor_y.sort_values(['rating', 'review_count'], ascending=False).head(10)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,coordinates.latitude,coordinates.longitude,location.address1,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address
22,5AabBubhVCZXHmBBuibFtg,arabesque-middle-eastern-foods-toronto,Arabesque Middle Eastern Foods,https://s3-media3.fl.yelpcdn.com/bphoto/p5J_Ko...,False,https://www.yelp.com/biz/arabesque-middle-east...,27,"[{'alias': 'mideastern', 'title': 'Middle East...",5.0,[],...,43.653070,-79.430960,1068 College Street,,,Toronto,M6H 1A9,CA,ON,"[1068 College Street, Toronto, ON M6H 1A9, Can..."
36,F1lBfCkoi3fYudixIMH2Sg,brewhaha-at-bandit-brewery-toronto,Brewhaha At Bandit Brewery,https://s3-media2.fl.yelpcdn.com/bphoto/6xFx5v...,False,https://www.yelp.com/biz/brewhaha-at-bandit-br...,15,"[{'alias': 'yelpevents', 'title': 'Yelp Events'}]",5.0,[],...,43.652500,-79.449570,2125 Dundas St W,None,,Toronto,M6R 1X1,CA,ON,"[2125 Dundas St W, Toronto, ON M6R 1X1, Canada]"
34,tyVRfb_Xr7r9aQ73Q-NQXQ,dufferin-grove-park-toronto,Dufferin Grove Park,https://s3-media4.fl.yelpcdn.com/bphoto/Gsfaqf...,False,https://www.yelp.com/biz/dufferin-grove-park-t...,14,"[{'alias': 'parks', 'title': 'Parks'}]",5.0,[],...,43.656462,-79.432504,875 Dufferin St,,,Toronto,M6H,CA,ON,"[875 Dufferin St, Toronto, ON M6H, Canada]"
45,4CDBiS3Q1JPk4WicswFPnA,de-floured-toronto-3,De Floured,https://s3-media4.fl.yelpcdn.com/bphoto/g8oXIK...,False,https://www.yelp.com/biz/de-floured-toronto-3?...,11,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",5.0,[],...,43.651570,-79.438350,1250 College Street,,,Toronto,M6H 1C2,CA,ON,"[1250 College Street, Toronto, ON M6H 1C2, Can..."
0,zEwQJl9haP5BYIl0peOpGA,sugo-toronto,Sugo,https://s3-media2.fl.yelpcdn.com/bphoto/5Fykw9...,False,https://www.yelp.com/biz/sugo-toronto?adjust_c...,199,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,[],...,43.658250,-79.442270,1281 Bloor Street W,,None,Toronto,M6H 1N7,CA,ON,"[1281 Bloor Street W, Toronto, ON M6H 1N7, Can..."
5,1DtyA7pnc66hTkv64lKlzw,takht-e-tavoos-toronto,Takht-e Tavoos,https://s3-media2.fl.yelpcdn.com/bphoto/3cEg4R...,False,https://www.yelp.com/biz/takht-e-tavoos-toront...,123,"[{'alias': 'persian', 'title': 'Persian/Irania...",4.5,[],...,43.652468,-79.432989,1120 College Street,,,Toronto,M6H 1B5,CA,ON,"[1120 College Street, Toronto, ON M6H 1B5, Can..."
9,HHvO4MgBFabsmlInou6viQ,le-baratin-toronto-5,Le Baratin,https://s3-media2.fl.yelpcdn.com/bphoto/xv3g5-...,False,https://www.yelp.com/biz/le-baratin-toronto-5?...,108,"[{'alias': 'french', 'title': 'French'}]",4.5,[],...,43.650070,-79.435100,1600 Dundas Street. W,None,,Toronto,M6K 1T8,CA,ON,"[1600 Dundas Street. W, Toronto, ON M6K 1T8, C..."
1,7DW2fsInJRfOHMTj2Z6fJw,brock-sandwich-toronto,Brock Sandwich,https://s3-media2.fl.yelpcdn.com/bphoto/7htxIK...,False,https://www.yelp.com/biz/brock-sandwich-toront...,101,"[{'alias': 'salad', 'title': 'Salad'}, {'alias...",4.5,[],...,43.658906,-79.440578,1260 Bloor Street W,,,Toronto,M6H 1N5,CA,ON,"[1260 Bloor Street W, Toronto, ON M6H 1N5, Can..."
4,H9pG0BNhqID2MTj8KnY3jQ,coo-café-bread-or-rice-toronto,COO Café Bread or Rice,https://s3-media4.fl.yelpcdn.com/bphoto/ZYXsQ1...,False,https://www.yelp.com/biz/coo-caf%C3%A9-bread-o...,84,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.5,[],...,43.660497,-79.431928,1049 Bloor Street W,,,Toronto,M6H 1M4,CA,ON,"[1049 Bloor Street W, Toronto, ON M6H 1M4, Can..."
15,mw69E3vLAbN-ra92f5Y8ZQ,black-cat-espresso-bar-toronto,Black Cat Espresso Bar,https://s3-media2.fl.yelpcdn.com/bphoto/XhdIio...,False,https://www.yelp.com/biz/black-cat-espresso-ba...,51,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.5,[],...,43.652780,-79.432350,1104 College Street,,,Toronto,M6H 1B3,CA,ON,"[1104 College Street, Toronto, ON M6H 1B3, Can..."


In [47]:
df_tokyo_y.sort_values(['rating', 'review_count'], ascending=False).head(10)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,coordinates.longitude,location.address1,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address,price
1,DtbufaMUdrX9pinKoH2Mqg,プリンチペ-デル-ソーレ-大田区,PRINCIPE del sole,https://s3-media2.fl.yelpcdn.com/bphoto/ahQhAC...,False,https://www.yelp.com/biz/%E3%83%97%E3%83%AA%E3...,2,"[{'alias': 'pizza', 'title': 'Pizza'}]",5.0,[],...,139.729162,東蒲田1-1-3,宝ビル 1F,,Ōta,144-0031,JP,13,"[東蒲田1-1-3, 宝ビル 1F, Ōta, 東京都 〒144-0031, Japan]",NaN
2,EuY4aWt9sOrFoBFdxZfuSg,満願堂-羽田空港店-大田区,Mangan-dō Haneda Airport,https://s3-media2.fl.yelpcdn.com/bphoto/bBH1l5...,False,https://www.yelp.com/biz/%E6%BA%80%E9%A1%98%E5...,1,"[{'alias': 'jpsweets', 'title': 'Japanese Swee...",5.0,[],...,139.742057,羽田空港3-4-2,第2旅客ターミナル 2F,,Ōta,144-0041,JP,13,"[羽田空港3-4-2, 第2旅客ターミナル 2F, Ōta, 東京都 〒144-0041, ...",￥￥
3,9_wSU7tFxpJSJeiLY_Ym4Q,貴船掘公園-大田区,貴船掘公園,https://s3-media1.fl.yelpcdn.com/bphoto/mNCWnQ...,False,https://www.yelp.com/biz/%E8%B2%B4%E8%88%B9%E6...,1,"[{'alias': 'parks', 'title': 'Parks'}]",5.0,[],...,139.741205,大森東3丁目,,,Ōta,143-0012,JP,13,"[大森東3丁目, Ōta, 東京都 〒143-0012, Japan]",NaN
6,EU3MSLkb4MXbJ0Ktv4a8GQ,ジャックニクラスゴルフセンター大森-大田区,ジャックニクラスゴルフセンター大森,https://s3-media3.fl.yelpcdn.com/bphoto/Bk2Lfq...,False,https://www.yelp.com/biz/%E3%82%B8%E3%83%A3%E3...,1,"[{'alias': 'golf', 'title': 'Golf'}]",5.0,[],...,139.739040,大森東3丁目28-1,None,None,Ōta,143-0012,JP,13,"[大森東3丁目28-1, Ōta, 東京都 〒143-0012, Japan]",NaN
19,GzkTHgzUd66A6nzOY9alQA,うめ屋敷-大田区,うめ屋敷,,False,https://www.yelp.com/biz/%E3%81%86%E3%82%81%E5...,1,"[{'alias': 'izakaya', 'title': 'Izakaya'}]",5.0,[],...,139.728808,東蒲田1-1-20,橋本ビル-1Ｆ-101,None,Ōta,144-0031,JP,13,"[東蒲田1-1-20, 橋本ビル-1Ｆ-101, Ōta, 東京都 〒144-0031, J...",NaN
0,n-Zay_vKhhOEFQjOF1B7Mg,大森ふるさとの浜辺公園-大田区,Oomori furusatonohamabe kouen,https://s3-media2.fl.yelpcdn.com/bphoto/R6dpQF...,False,https://www.yelp.com/biz/%E5%A4%A7%E6%A3%AE%E3...,5,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,[],...,139.741485,ふるさとの浜辺公園1-1,,,Ōta,,JP,13,"[ふるさとの浜辺公園1-1, Ōta, 東京都, Japan]",NaN
4,mWR5xjHZyMB4L4_KrdtNZg,たぐい食堂-大田区,Taguishokudō,https://s3-media1.fl.yelpcdn.com/bphoto/orNn-d...,False,https://www.yelp.com/biz/%E3%81%9F%E3%81%90%E3...,2,"[{'alias': 'onigiri', 'title': 'Onigiri'}]",4.5,[],...,139.731644,大森西3-21-5,,,Ōta,143-0015,JP,13,"[大森西3-21-5, Ōta, 東京都 〒143-0015, Japan]",NaN
5,vWhLjGeJ7VhrpsDGmWTDOQ,タケウチ製パン-大田区,タケウチ製パン,https://s3-media4.fl.yelpcdn.com/bphoto/iRXd6a...,False,https://www.yelp.com/biz/%E3%82%BF%E3%82%B1%E3...,1,"[{'alias': 'bakeries', 'title': 'Bakeries'}]",4.0,[],...,139.735444,大森中2丁目15-3,None,None,Ōta,143-0014,JP,13,"[大森中2丁目15-3, Ōta, 東京都 〒143-0014, Japan]",NaN
7,UWMjlEC19FhowOBibLeWxw,串カツ田中-大森店-大田区,Kushikatsu tanaka Oomori,https://s3-media2.fl.yelpcdn.com/bphoto/b31HSu...,False,https://www.yelp.com/biz/%E4%B8%B2%E3%82%AB%E3...,1,"[{'alias': 'kushikatsu', 'title': 'Kushikatsu'}]",4.0,[],...,139.742057,大森北一丁目38-2,,,Ōta,143-0016,JP,13,"[大森北一丁目38-2, Ōta, 東京都 〒143-0016, Japan]",NaN
8,nQGKKuH-rkQVNuMvb7xf9g,大田区立貴船堀緑地-大田区,Ootakuritsukibunehoriryokuchi,https://s3-media1.fl.yelpcdn.com/bphoto/DPCzLD...,False,https://www.yelp.com/biz/%E5%A4%A7%E7%94%B0%E5...,1,"[{'alias': 'parks', 'title': 'Parks'}]",4.0,[],...,139.741947,大森東3丁目27,,,Ōta,143-0012,JP,13,"[大森東3丁目27, Ōta, 東京都 〒143-0012, Japan]",NaN
